In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data_file/air_sensor001_data.csv")
df = df.drop(columns = 'Unnamed: 0')
df = df.drop(columns = 'csq')
df = df.drop(columns = 'deviceid')
df

,eco2,ethanol,h2,pm10,pm100,pm25,rh,temp,voc,Datetime
0,634,18727,12998,5,7,6,51,25,288,2022-05-01 04:29:00
1,606,18710,13023,5,7,7,51,25,296,2022-05-01 04:30:00
2,715,18732,12936,6,8,8,51,25,279,2022-05-01 04:31:00
3,699,18690,12948,5,6,6,51,25,311,2022-05-01 04:32:00
4,695,18737,12958,5,6,6,51,25,273,2022-05-01 04:33:00
...,...,...,...,...,...,...,...,...,...,...
22961,825,18780,12898,3,5,5,52,24,306,2022-05-17 03:10:00
22962,855,18772,12875,3,6,4,51,24,313,2022-05-17 03:11:00
22963,822,18783,12909,3,5,4,51,24,321,2022-05-17 03:12:00
22964,776,18819,12942,3,4,3,52,24,276,2022-05-17 03:13:00


In [3]:
data_1 = df[["Datetime","pm25","pm10"]]
data_1 = data_1.set_index('Datetime')
#data_1

In [4]:
data_1["pm25_12hr_avg"] = data_1["pm25"].rolling(window = 12*60, min_periods = 6*60).mean().values
data_1["pm10_12hr_avg"] = data_1["pm10"].rolling(window = 12*60, min_periods = 6*60).mean().values
data_1["pm25_12hr_avg"] = data_1["pm25_12hr_avg"].fillna(0)
data_1["pm10_12hr_avg"] = data_1["pm10_12hr_avg"].fillna(0)
#data_1

In [5]:
data_1["pm25_4hr_avg"] = data_1["pm25"].rolling(window = 4*60, min_periods = 2*60).mean().values
data_1["pm10_4hr_avg"] = data_1["pm10"].rolling(window = 4*60, min_periods = 2*60).mean().values
data_1["pm25_4hr_avg"] = data_1["pm25_4hr_avg"].fillna(0)
data_1["pm10_4hr_avg"] = data_1["pm10_4hr_avg"].fillna(0)
#data_1

In [6]:
data_1["pm25_instant"] = (data_1["pm25_12hr_avg"]+data_1["pm25_4hr_avg"])*0.5
data_1["pm10_instant"] = (data_1["pm10_12hr_avg"]+data_1["pm10_4hr_avg"])*0.5
data_1 = data_1.drop(columns = 'pm25_12hr_avg')
data_1 = data_1.drop(columns = 'pm10_12hr_avg')
data_1 = data_1.drop(columns = 'pm25_4hr_avg')
data_1 = data_1.drop(columns = 'pm10_4hr_avg')
#data_1

In [7]:
data_1["AQI"] = data_1[["pm25_instant","pm10_instant"]].max(axis=1)
#data_1

In [8]:
def C_F_Convert(temp,measurement_units):
    if measurement_units == "C" :
        Fahrenheit = (temp * 1.8) + 32
        return Fahrenheit
    elif measurement_units == "F" :
        Celsius = (temp - 32) /1.8
        return Celsius
    else : 
        return "error"

In [9]:
def heat_index_formula(F_temp,relative_humidity):
    c1 = -42.379
    c2 = 2.04901523
    c3 = 10.14333127
    c4 = -0.22475541
    c5 = -6.83783e-3
    c6 = -5.481717e-2
    c7 = 1.22874e-3
    c8 = 8.5282e-4
    c9 = -1.99e-6
    
    T = F_temp
    R = relative_humidity
    
    HI = c1 + c2*T + c3*R + c4*T*R + c5*T*T + c6*R*R + c7*T*T*R + c8*T*R*R + c9*T*T*R*R
    return HI

In [10]:
data_2 = df[["Datetime","temp","rh"]]
data_2 = data_2.set_index('Datetime')
#data_2

In [11]:
data_2['temp_F'] = C_F_Convert(data_2['temp'],"C")
data_2['HI_F'] = heat_index_formula(data_2['temp_F'],data_2['rh'])
data_2['HI'] = C_F_Convert(data_2['HI_F'],"F")
data_2 = data_2.drop(columns = 'temp_F')
data_2 = data_2.drop(columns = 'HI_F')
data_2

,temp,rh,HI
Datetime,,,
2022-05-01 04:29:00,25,51,25.903654
2022-05-01 04:30:00,25,51,25.903654
2022-05-01 04:31:00,25,51,25.903654
2022-05-01 04:32:00,25,51,25.903654
2022-05-01 04:33:00,25,51,25.903654
...,...,...,...
2022-05-17 03:10:00,24,52,25.384546
2022-05-17 03:11:00,24,51,25.389758
2022-05-17 03:12:00,24,51,25.389758


In [12]:
# co2
# 350<新鮮<450<可接受<700<不舒服(開啟換氣)<800<(加大換氣量)<1000<換氣量太低<2500<對健康不利<5000<盡量避免

# https://iaq.epa.gov.tw/indoorair/improvement_air.aspx
# https://sites.google.com/site/judaoshineikongqipinzhiwang/huan-bao-shu-yu-gao-ding-ding-ying-fu-he-shi-nei-kong-qi-pin-zhi-guan-li-fa-zhi-di-er-pi-gong-gao-chang-suo-cao-an/zui-jin-xiao-xi/shineic02nongdufeiguanbuke